# Feature evaluation

org: **D4G**  project: **BGCO**  task: **feature evaluation**

data: labeled (engage vs. static) dataset highlighting changes in member engagement (visits per week) between first and year of engagement

## Prep the environment

In [ ]:
import pandas as pd  # used to manipulate dataframes
import numpy as np # used to manipulate series

import seaborn as sns  # needed for visualing
import matplotlib.pyplot as plt  # needed for visualing

## Data Wrangling

### Load Data

In [ ]:
# read in the dataframe
df_001 = pd.read_csv("D4G_BGCO_Churn_Labeled.csv", encoding = "cp1252")

### Feature Evaluation

In [ ]:
df_002 = df_001.drop(droplist_unique, axis = 1)  # features with no unique values removed

In [ ]:
df_003 = df_002.drop(['member_identity', 'birth_year', 'age', 'city', 'active', 'd4g_valid_postal_code', 'checkin_avg', 'grade', 'age_category'], axis = 1)  # redundant features

In [ ]:
df_004 = df_003.drop(['member_number', 'd4g_school_id'], axis = 1)  # erroneous aggregation, school_id has many -1

In [ ]:
df_005 = df_004.drop(droplist_missing, axis = 1)  # missing values exceed 20%

In [ ]:
df_006 = df_005.drop(imbalance, axis = 1)  # where one value count > 85%

In [ ]:
df = df_006

In [ ]:
print("The training set has {0} rows and {1} columns".format(df.shape[0], df.shape[1]))

In [ ]:
df.to_csv("D4G_BGCO_Churn_Labeled_ForEDA.csv", index = False)

### Feature Summary

Proposed
* total number of visits to clubhouse in y1 (find out if clubhouse is a factor)
* visits per week or month to clubhouse in y1 (find out if clubhouse is a factor)

Key
* d4g_member_id (primary key)

Behaviour
* label (target feature)
* member_fees_total
* total number of visits by member in y1
* visits grouped by fall or winter

Service
* first_year: the first year of engagement
* member_location: clubhouse visited

Characteristic
* Y1_Age: age at first_year
* age category
* sex
* postal_code (see Conditional Inclusions)
* d4g_school_id

Conditional Inclusions
* first_language (only 60% of the dataset) (missing data)
* postal_code (only 90% of the dataset) (missing data)
* d4g_valid_postal_code (only 85% of the dataset) (missing data) (more accurate?)


### Unique Values

Number of unique values, if 1 or less should be removed.

In [ ]:
df.nunique()

Create a list of columns with low unique value counts

In [ ]:
droplist_unique = [i for i in df.columns if df[i].nunique('label') < 2]
droplist_unique  # from df_001

### Erroneous Aggregation

Special case: Member numbers
* the duplicates in this feature seem to correspond to different members
* thus inclusion erroneously reduces variability, without adding qualifiers

In [ ]:
duplicates = df.duplicated('member_number', keep = False)
df_duplicates = df[duplicates]
df_duplicates[['member_number','first_year', 'sex', 'Y1_Age']].sort_values(by = 'member_number').head(6)

### Missing Values

How many features/attributes have missing values? Which ones?
* df_004 used to exclude if missing > 20%

In [ ]:
print("A total of ", len(df.columns[df.isnull().any()]), "features have missing values")
print("They are:", df.columns[df.isnull().any()])

How many are missing?

In [ ]:
miss = df.isnull().sum() #number of missing entries in each feature / number of total entries
miss = miss[miss > 0] # keep only those that are greater than "0 / number of total entries"
miss.sort_values(inplace=True) # sort by percentage ascending
miss # display

What is the percentage of missing values?

In [ ]:
miss = df.isnull().sum()/len(df) #number of missing entries in each feature / number of total entries
miss = miss[miss > 0] # keep only those that are greater than "0 / number of total entries"
miss.sort_values(inplace=True) # sort by percentage ascending
miss # display

Visualising missing values

In [ ]:
miss = miss.to_frame() # convert to a dataframe
miss.columns = ['count'] # rename the column as 'count'
miss.index.names = ['Name'] # rename index as 'Name'
miss['Name'] = miss.index # create a new column of the index

#plot the missing value count
sns.set(style="whitegrid", color_codes=True)
sns.barplot(x = 'Name', y = 'count', data=miss)
plt.xticks(rotation = 90)
plt.show()

Remove if exceeds 25%

In [ ]:
droplist_missing = [i for i in df.columns if df[i].isnull().sum()/len(df[i]) > 0.2]
droplist_missing  # from df_004

### Feature Balance

Create a smaller subset of 

In [ ]:
balance_check = [i for i in df.columns if df[i].nunique() < 50]
df_balance_check = df[balance_check]

df_balance_check.nunique()

Define a function to remove columns that have a value count exceeding 90%

In [ ]:
def imbalanced_features(df_to_check, threshold = 0.9):
    """
    Returns a list of columns where at least one value count exceeds threshold
    """
    
    df_to_output = df_to_check
    
    for h in df_to_check.columns:
        col_val_count = pd.value_counts(df_to_check[h].values, sort=False)  # returns value counts by column
        
        count = 0
        
        for i in col_val_count.values:
            if i / df_to_check.shape[0] > threshold:
                count += 1
        
        if count > 0:
            df_to_output = df_to_output.drop(h, axis = 1)
            
    list_to_output = df_to_check.drop(df_to_output.columns, axis = 1).columns
            
    return list_to_output

Use function to create list of imbalanced features

In [ ]:
imbalance = imbalanced_features(df_balance_check, 0.85)
imbalance

Update df by dropping imbalanced features

In [ ]:
categorical_features = df_balance_check.drop(imbalance, axis = 1).columns

In [ ]:
# plot parameters
ROWS, COLS = 3, 3  # set number of rows and columns for figures

fig, ax = plt.subplots(ROWS, COLS, figsize=(18, 9))
row, col = 0, 0
for i, categorical_feature in enumerate(categorical_features):
    if col == COLS - 1:
        row += 1
    col = i % COLS
    df[categorical_feature].value_counts().plot('bar', ax=ax[row, col]).set_title(categorical_feature)